# to make the match for the Table 2 1993 QS fe_13 data

## this notebook recreates the basic data used in 

## Dere, 2020, MNRAS, 496, 2334

In [1]:
import json
import pickle
import fnmatch
import matplotlib.pyplot as plt
import numpy as np
import ChiantiPy.core as ch
import ChiantiPy.tools.util as chutil
import ChiantiPy.model as mdl
import ChiantiPy.model.Maker as mm
import ChiantiPy.Gui as chGui

In [2]:
%autoreload 3

In [3]:
matplotlib qt

In [4]:
myIon = 'fe_13'

In [5]:
nameDict = chutil.convertName(myIon)

In [6]:
nameDict.keys()

dict_keys(['Z', 'Ion', 'Dielectronic', 'Element', 'higher', 'lower', 'filename', 'iso', 'isoEl', 'spectroscopic', 'experimental'])

In [7]:
nameDict['spectroscopic']

'Fe XIII'

In [8]:
nameDict['experimental']

'Fe 12+'

In [9]:
ls

0.test.notebooks.in.git.dir
1_fe_13_demo_make_match.html
1_fe_13_demo_make_match.ipynb
2_fe_13_demo_check_model.html
2_fe_13_demo_check_model.ipynb
3_fe_13_demo_chi2_search.ipynb
4_fe_13_demo_mcmc.ipynb
5_fe_13_demo_mcmc_trace_analyze.ipynb
fe_13_demo_make_model.html
predictPrintPrior.txt
results_2022_March_28_2000_100000.pkl
tab2_1993_qs_fe_13_1dsearch.pkl
tab2_1993_qs_fe_13_1dsearch_summary.txt
tab2_1993_qs_fe_13.json
tab2_1993_qs_fe_13_match.pkl
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-21_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-22_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-28_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-30_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-21_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-22_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-28_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_1

## fe_13 intensities are in the .json file

In [10]:
with open('tab2_1993_qs_fe_13.json', 'r') as inpt:
    specData = json.load(inpt)

In [11]:
specData.keys()

dict_keys(['filename', 'wvl0', 'intensity', 'intStd', 'dwvl', 'ions', 'ref'])

In [12]:
specData['dwvl']

[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04]

## the model.Maker will match any CHIANTI lines to those in the specData that are with 'dwvl'

In [13]:
specData['ref']

['Brosius, Jeffrey W.; Davila, Joseph M.; Thomas, Roger J.; Monsignori-Fossi, Brunella C.',
 'Measuring Active and Quiet-Sun Coronal Plasma Properties with Extreme-Ultraviolet Spectra from SERTS ',
 '1996, Astrophysical Journal Supplement v.106, p.143',
 'ADSref:  https://ui.adsabs.harvard.edu/abs/1996ApJS..106..143B/abstract']

In [14]:
temp = 1.78e+6
dens  = 10.**(6.+ 0.025*np.arange(241))

## chi-squared = sum ((observed -  predicted)/(wghtFactor * observed))^2

In [15]:
wghtFactor = 0.2

In [16]:
box = mm.maker(specData, temp, dens, wghtFactor = wghtFactor, ionList=['fe_13'], verbose = True)

 XUVTOP = /data2/a2git/chdbase/
 abundanceName = sun_photospheric_2015_scott
 minimum abundance =   1.00e+01
 # of observables / reduce # = 9        3.00


## the next step matches the observed lines to the CHIANTI lines to within tab['dwvl']

## in this demo, only searching for fe_13 matches

In [17]:
box.makeMatch(verbose=True)

 fe_13 in the CHIANTI database
 in makeMatchGate thision = fe_13


## next, calculate the contribution functions (gofnt) for each line, including any blends

## with more ions, the multiprocessing function mgofnt(temp, dens) is faster

In [18]:
box.gofnt(temp, dens, verbose=True)

 temperature size:    241
 density     size:    241
 using fe_13
 elapsed seconds =       40.000


## save the match attribute as a pickle file to be picked on again.

In [19]:
matchFilename = 'tab2_1993_qs_fe_13_match.pkl'

In [20]:
box.saveMatch(filename=matchFilename )

NT not available


In [21]:
ls

0.test.notebooks.in.git.dir
1_fe_13_demo_make_match.html
1_fe_13_demo_make_match.ipynb
2_fe_13_demo_check_model.html
2_fe_13_demo_check_model.ipynb
3_fe_13_demo_chi2_search.ipynb
4_fe_13_demo_mcmc.ipynb
5_fe_13_demo_mcmc_trace_analyze.ipynb
fe_13_demo_make_model.html
predictPrintPrior.txt
results_2022_March_28_2000_100000.pkl
tab2_1993_qs_fe_13_1dsearch.pkl
tab2_1993_qs_fe_13_1dsearch_summary.txt
tab2_1993_qs_fe_13.json
tab2_1993_qs_fe_13_match.pkl
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-21_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-22_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-28_wvl.txt
tab2_1993_qs_fe_13_match_postDiffPrint_2000_100000_2022-03-30_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-21_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-22_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_100000_2022-03-28_wvl.txt
tab2_1993_qs_fe_13_match_postPredictPrint_2000_1

## the next 5 steps if you want to restart without recalculating the match attribute

## the pickle file has already been created

In [22]:
mydir = os.getcwd()
mylist = os.listdir('.')
newlist = fnmatch.filter(mylist,'*.pkl')
selected = chGui.gui.selectorDialog(newlist,'pick one')

In [23]:
mypkl = selected.selectedText[0]

In [24]:
print(mypkl)

tab2_1993_qs_fe_13_match.pkl


In [25]:
with open(mypkl,'rb') as inpt:
    match = pickle.load(inpt)

In [26]:
match.keys()

dict_keys(['match', 'Temperature', 'EDensity', 'Ndens', 'Ntemp', 'NTempDens', 'MinAbund', 'WghtFactor'])

## the next two steps if you want to restart without recalculating the match attribute

In [27]:
box = mm.maker(specData)

 XUVTOP = /data2/a2git/chdbase/
 abundanceName = sun_photospheric_2015_scott
 minimum abundance =   1.00e+01
 # of observables / reduce # = 9        3.00


In [28]:
box.loadMatch(mypkl)

 EmIndices not in matchDict
Em not in matchDict
Em not in matchDict
NT not in matchDict


## can just continue here

## plot the contribution functions vs density

In [29]:
temp = box.Temperature
' T:  %12.2e'%(temp[0])

' T:      1.78e+06'

In [30]:
dens = box.EDensity
' Ne:  %12.2e'%(dens[0])

' Ne:      1.00e+06'

In [31]:
nMatches = len(box.Match)
print('%3i'%(nMatches))

  9


## adjust and lr is just to get a pretty plot

In [32]:
adjust = np.ones(nMatches,np.float64)

### adjust is an array to multplicatively adjust the heights of the labels

In [33]:
adjust[2] = 0.70
adjust[3] = 1.05
adjust[4] = 0.70
adjust[8] = 0.75

### lr could be an array of cantaining either 'b', for both, 'n', for none, 'l' for left, 'r', for right

In [34]:
lr = ['l','l','l','r','l','r','r','l','r']

In [35]:
print(' len of lr:  %5i needs to be the same as the wavelength list %i'%(len(lr), len(specData[''])))

TypeError: not enough arguments for format string

In [ ]:
plt.figure()
box.emPlot(vs='d', fs=12, loc='center right', adjust = adjust, position = lr)

In [ ]:
left,right = plt.xlim()
bottom, top = plt.ylim()
plt.xlim(left, 1.e+14)
plt.ylim(bottom, 1.e+28)
plt.text(1.e+8, 3.e+26, nameDict['spectroscopic'], fontsize=18)
plt.tight_layout()

## alternatively, there is a object oriented emPlotObj

In [ ]:
fig, ax = box.emPlotObj(vs='d', fs=12, loc='center right', adjust = adjust, position = lr)

## it is possible to continue to adjust things

In [ ]:
ax.set_title('mytitle', fontsize=14)

In [ ]:
fig.tight_layout()